In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})
import time


# CMPS 2200
# Introduction to Algorithms

## Recurrences


Recurrences are a way to capture the behavior of recursive algorithms.

Key ingredients: 
- Base case ($n = c$): constant time 
- Inductive case ($n > c$): recurse on smaller instance and use output to compute solution
```python
def factorial(x):
    if x == 1:
        return x
    else:
        return (x * factorial(x-1))
```

Actually recursion is a conceptual way to view algorithm execution, and we can reframe an algorithm specification to make it recursive.


In [2]:
def selection_sort(L):
    for i in range(len(L)):
        print(L)
        m = L.index(min(L[i:]))
        L[i], L[m] = L[m], L[i]
    return L
                   
selection_sort([2, 1, 4, 3, 9])

[2, 1, 4, 3, 9]
[1, 2, 4, 3, 9]
[1, 2, 4, 3, 9]
[1, 2, 3, 4, 9]
[1, 2, 3, 4, 9]


[1, 2, 3, 4, 9]

In [3]:
def selection_sort_recursive(L):
    print('L=%s' % L)
    if (len(L) == 1):
        return(L)
    else:
        m = L.index(min(L))
        L[0], L[m] = L[m], L[0]
        return    selection_sort_recursive(L[1:])
    
selection_sort_recursive([2, 1, 999, 4, 3])

L=[2, 1, 999, 4, 3]
L=[2, 999, 4, 3]
L=[999, 4, 3]
L=[4, 999]
L=[999]


[999]

Are these the same algorithm? Can we give a SPARC specification?

<p><span class="math display">\[\begin{array}{l}  
\mathit{selectionsort}~~L = 
\\  
~~~~~~~~~\texttt{if}~|L| = 1~\texttt{then}    
\\  
~~~~~~~~~~~~~\texttt{return}~~L  
\\  
~~~~~~~~~\texttt{else}
\\
~~~~~~~~~~~~\texttt{let}\\
~~~~~~~~~~~~~~~m = \texttt{minimum element in}~~L\\
~~~~~~~~~~~~\texttt{in}\\ 
~~~~~~~~~~~~~~~\texttt{Cons}(m, (\mathit{selectionsort~~\langle x | x\in L~~and~~x\neq m \rangle})) \\
~~~~~~~~~~~~\texttt{end} 
\end{array}\]</span></p>



What is the Work $W(n)$ and Span $S(n)$ given the list with length $n$ ($n=1, 2, 3, \cdots$)?

$\begin{eqnarray}
W(n) &=& W(n-1) + n 
\end{eqnarray}$


$\begin{eqnarray}
W(n) &=& W(n-2) + (n-1) + n\\
 & = & W(n-3)+(n-2)+(n-1) + n\\
&\vdots&
\end{eqnarray}$

$\begin{eqnarray}
W(n) &=& \sum_{i=1}^n i  \\
&=& \frac{n(n+1)}{2}  = O(n^2).
\end{eqnarray}$


The recurrence for Selection Sort is somewhat simple - what if we have multiple recursive calls and split the input? (This is actually what *divide-and-conquer* algorithms do.)

We'll look at methods to solve recurrences in order to obtain **big-O** bounds for recursive algorithms.

We will:
- Get intuition for recurrences by looking the recursion **tree**. 

- Develop the **brick** method to quickly state asymptotic bounds on a recurrence by looking at the shape of the tree.

Let's look at the specification and recurrence for Merge Sort: 

<p><span class="math display">\[\begin{array}{l}  
\mathit{mergeSort}~a =  
\\   
~~~~\texttt{if}~|a| \leq 1~\texttt{then}  
\\   
~~~~~~~~a  
\\  
~~~~\texttt{else}  
\\   
~~~~~~~~\texttt{let}  
\\  
~~~~~~~~~~~~(l,r) = \mathit{splitMid}~a  
\\   
~~~~~~~~~~~~(l',r') = (\mathit{mergeSort}~l \mid\mid{} \mathit{mergeSort}~r)  
\\  
~~~~~~~~\texttt{in}  
\\   
~~~~~~~~~~~~\mathit{merge} (l',r')  
\\  
~~~~~~~~\texttt{end}  
\end{array}\]</span></p>

Suppose that the merging step can be done with $O(n)$ work and $O(\log n)$ span. Then recurrence for the work is: 

$ \begin{equation}
W(n) = \begin{cases}
  c_b, & \text{if $n=1$} \\
  2W(n/2) + O(n) = 2W(n/2) + O(n), & \text{otherwise} 
  \end{cases}
\end{equation}$

How do we solve this recurrence to obtain $W(n) = O(n\log n)$?





![alttext](figures/mergesort_tree.png)

The recursion tree for Merge Sort has linear work at every level except at the leaves. There are a logarithmic number of levels and a linear number of leaves so we obtain an asymptotic bound of $O(n\log n)$ for the work.

## Solving Recurrences with the Tree Method 

<br>


<div>size at level $i$</div> <div style="text-align: right"> cost at level $i$ </div>

![merge-tree.jpg](figures/merge-tree.jpg)

### Recipe: 
1. Determine the cost of each level $i$ ($i$ starts at $0$).
2. Determine the number of levels
3. Cost = $\sum_{i=0}^{\hbox{num levels}}$ cost for level $i$
  - This last step usually involves using properties of series
  
<br>

E.g., for merge sort:

- level $i$ contains $2^i$ nodes
- each node at level $i$ costs $c_1 \frac{n}{2^i} + c_2$
- so, each level costs $2^i * (c_1 \frac{n}{2^i} + c_2) = c_1n + 2^i c_2  $
- since each level reduces size by half, we have $\log_2 n$ levels
- so, total cost of tree is:

$$W(n) = \sum_{i=0}^{\log_2 n} (c_1n + 2^i c_2) = \sum_{i=0}^{\log_2 n}c_1 n + \sum_{i=0}^{\log_2 n} 2^i c_2$$

To solve this, we'll make use of bounds for **geometric series**. 

For $\alpha > 1$: 
$\:\:\: \sum_{i=0}^n \alpha^i <\frac{\alpha}{\alpha - 1}\cdot\alpha^n$

> e.g., $\sum_{i=0}^{\log_2 n} 2^i < \frac{2}{1} * 2^{\log_2 n} = 2n$

For $\alpha < 1$: 
$\:\:\: \sum_{i=0}^\infty \alpha^i  < \frac{1}{1-\alpha}$

> e.g., $\sum_{i=0}^{\log_2 n} \frac{1}{2^i} < 2$


<br> plugging in...

<!-- $$= \sum_{i=0}^{\log_2 n} (c_1 n + 2^i c_2)$$ -->

$$= \sum_{i=0}^{\log_2 n}c_1 n + \sum_{i=0}^{\log_2 n} 2^i c_2$$

$$= c_1n \sum_{i=0}^{\log_2 n} 1 + c_2 \sum_{i=0}^{\log_2 n} 2^i$$

$$<c_1n \log_2 n + 2 c_2 n$$

$$\in O(n \log_2 n)$$ [Base 2 is omitted.]

What about the span?

![alttext](figures/tree.png)


The recurrence for the span of Mergesort is:

$ \begin{equation}
S(n) = \begin{cases}
  c_3, & \text{if $n=1$} \\
  S(n/2) + c_4 \lg n, & \text{otherwise} 
  \end{cases}
\end{equation}$


Since each level of the recursion tree is concurrent and all nodes have the same cost, we have that

$ \begin{align}
S(n) & = \sum_{i=0}^{\lg n} \lg\frac{n}{2^i}\\
& = \sum_{i=0}^{\lg n} (\lg n - i)\\
& = \sum_{i=0}^{\lg n} (\lg n) - \sum_{i=1}^{\lg n} i\\
& = \lg n * (\lg n+1)  - \frac{1}{2}\lg n * (\lg n+1) \:\: (\hbox{using}\:\:\sum_{i=1}^n = \frac{n(n+1)}{2})\\
& = \frac{1}{2}\lg^2 n + \frac{1}{2}  \lg n\\
& \in O(\lg^2 n)\\
\end{align}$


Another recurrence:
    
$ \begin{equation}
W(n) = \begin{cases}
  c_b, & \text{if $n=1$} \\
  2W(n/2) + O(n^2), & \text{otherwise} 
  \end{cases}
\end{equation}$

What is the asymptotic runtime?

![alttext](figures/tree.png)



$$W(n) = 2W(n/2) + c_1n^2 + c_2$$

<img width="70%" src="figures/n_squared.png"/>

$= \sum_{i=0}^{\lg n} (c_1 \frac{n^2}{2^i} + 2^i c_2)$

$= c_1 n^2 \sum_{i=0}^{\lg n} \frac{1}{2^i} + c_2 \sum_{i=0}^{\lg n} 2^i$

$< 2 c_1 n^2 + 2 c_2 n$

$\in O(n^2)$

So what if branching factor is not 2?

$$W(n) = 4 W \Big(\frac{n}{2}\Big) + O(n)$$

**costs**

 level 0: $c_1n + c_2$
 
 level 1: $4(c_1 \frac{n}{2} + c_2)$
 
 level 2: $16(c_1 \frac{n}{4} + c_2)$
 
 level $i$ ?

$$4^i(c_1 \frac{n}{2^i} + c_2)$$

<br>

still $\lg n$ levels:, so $W(n)$ is:

<br>

$$= c_1n \sum_{i=0}^{\lg n} \Big(\frac{4}{2}\Big)^i + c_2 \sum_{i=0}^{\lg n} 4^i$$

$$< 2 c_1 n^2 + \frac{4}{3} c_2 4^{\lg n}$$

$$(\hbox{since} \:\:\ \sum_{i=0}^n \alpha^i  < \frac{\alpha}{\alpha - 1}\cdot\alpha^n)$$

$$= 2 c_1 n^2 + \frac{4}{3} c_2 2^{\lg n} 2^{\lg n}$$

$$= 2 c_1 n^2 + \frac{4}{3} c_2 n^2$$

$$\in O(n^2)$$

Recurrences can be more general, so how do we get a handle on asymptotic runtimes when the recursion is really complicated?

**Key Idea:** The branching properties of the recursion tree determine work at each level and the number of leaves.

The **brick method** gives a way to derive asymptotic runtimes by looking at the relationships between parent and child nodes in the recursion tree. This way we only need to worry about the costs of the root and the leaves.

> In the tree method, if the costs grow or decay geometrically across levels, then we need only consider the cost of the root (**decay**), or the total cost of the leaves (**growth**). 

> If there is no geometric growth or decay then we can calculate the cost of the worst level (often either the root or leaves) and multiply it by the number of levels. 

This leads to three cases:

1. **root dominated**
2. **leaf dominated**
3. **balanced**

> Conveniently, to distinguish these three cases we need only consider the cost of each node in the tree and how it relates to the cost of its children.

The value of $n$ decreases geometrically as we collect the terms in our recurrences. We'll make use of bounds for geometric series. 

For any $\alpha > 1$:
    
$$ \sum_{i=0}^n \alpha^i  = \frac{\alpha^{n+1}-1}{\alpha - 1} < \frac{\alpha}{\alpha - 1}\cdot\alpha^n$$

For $\alpha < 1$:

$$ \sum_{i=0}^\infty \alpha^i  = \frac{1-\alpha^{n+1}}{1-\alpha} < \frac{1}{1-\alpha}$$



### Root-dominated

For a node $v$ in the recursion tree, let $C(v)$ denote its cost and $D(v)$ denote its children.

A recurrence is **root-dominated** if for all $v$, there is an $\alpha > 1$ such that:

$$C(v) \geq \alpha \sum_{u \in D(v)} C(u)$$

The cost of a root dominated recurrence is $O(C(r))$ if $r$ is the root.

This is because the cost reduces geometrically as we go toward the leaves, and the total cost bounded by $\alpha/(\alpha-1)$ times $C(r)$.


$$W(n) = 2 W\Big(\frac{n}{2}\Big) + n^2$$

$C(\hbox{root}) = n^2$

$C(\hbox{level}\:1) = \Big(\frac{n}{2}\Big)^2 + \Big(\frac{n}{2}\Big)^2 = \frac{n^2}{2}$

<br>

Cost has **decreased** by a factor of two when descending one level in the tree.

<br>

so, if $\alpha \leftarrow 2$:

$C(v) \ge \alpha \sum_{u \in D(v)} C(u)$

$n^2 \ge 2 * \frac{n^2}{2}$

$n^2 \ge n^2$

<br>

Because the cost is asymptotically dominated by the **root**, we need to only consider its cost: $O(n^2)$.


### Leaf-dominated

A recurrence is **leaf-dominated** if for all $v$, there is an $\alpha > 1$ such that:

$$C(v) \leq \frac{1}{\alpha} \sum_{u \in D(v)} C(u)$$

If we have $L$ leaves in the recursion tree, the cost of a leaf dominated recurrence is $O(L)$.

This is because the cost increases geometrically as we go toward the leaves, and the total cost is bounded by $\alpha/(\alpha-1)$ times $c_b \cdot L$.



$$W(n) = 2W\Big(\frac{n}{2}\Big) + \sqrt{n}$$


$C(\hbox{root}) = \sqrt{n}$

$C(\hbox{level}\:1) = \sqrt{\Big(\frac{n}{2}\Big)} + \sqrt{\Big(\frac{n}{2}\Big)} = 2 \frac{\sqrt{n}}{\sqrt{2}} = \sqrt{2}\sqrt{n}$

<br>

Cost has **increased** by a factor of $\sqrt{2}$ when descending one level in the tree.

<br>

so, if $\alpha \leftarrow \frac{1}{\sqrt{2}}$:

$C(v) \le \frac{1}{\alpha} \sum_{u \in D(v)} C(u)$

$\sqrt{n} \le \frac{1}{\sqrt{2}} * \sqrt{2}\sqrt{n}$

$\sqrt{n} \le \sqrt{n}$

<br>

Because the cost is asymptotically dominated by the **leaves** (i.e., lowest level of the tree), we need to only consider their cost:

<br>

- Cost of each leaf is 1 (since $\sqrt{1}==1$).
- Depth of tree is $\lg{n}$ (since we divide by 2 at each level)
- Number of leaves of a binary tree with depth $\lg{n}$ is $2^{\lg{n}} = n$

<br>
So, final cost is $n*1 = O(n)$


### Balanced

A recurrence is **balanced** when every level of the recursion tree has the same asymptotic cost. In this case, the recurrence is
*number of levels* times *maximum cost per level*.



$O(D(r) \log n) = O(L \log n)$.  


$$W(n) = 2 W\Big(\frac{n}{2}\Big) + n$$

$C(\hbox{root}) = n$

$C(\hbox{level}\:1) = \Big(\frac{n}{2}\Big) + \Big(\frac{n}{2}\Big) = n$

<br>

Cost has **remained the same** when descending one level in the tree.

So, total cost is *number of levels* times *maximum cost per level*.

<br>

- Number of levels = $\lg n$
- Maximum cost per level = $n\:\:$ (last level: $n$ leaves with cost 1 each; first level: one node with cost $n$)
- $O(n \lg n)$


Let's look at some examples:

$$ W(n) = 3 W(n/2) + n $$

$$ W(n) = 2 W(n/3) + n $$

$$ W(n) = 3 W(n/3) + n $$

Do you see a way to count the number of leaves in the recursion tree?


>Useful observation: If we have a recurrence of the form $W(n) = aW(n/b) + f(n)$, then the number of leaves is $O(a^{\log_b n})$, or equivalently, $O(n^{log_b a})$.

$$ W(n) = 3 W(n/2) + n $$

This is leaf-dominated, so it is $O(n^{\log_2 3})$.

$$ W(n) = 2 W(n/3) + n $$

This is root-dominated, so it is $O(n)$.

$$ W(n) = 3 W(n/3) + n $$

This is balanced, so it is $O(n \log n)$.


More examples (some trickier than others):

$$ W(n) = W(n - 1) + n $$

$$ W(n) = \sqrt{n} W(\sqrt{n}) + n^2 $$

$$ W(n) = W(\sqrt{n}) + W(n/2) + n $$

$$ W(n) = W(n/2) + W(n/3) + 1 $$

